# Sorting and fast-track alogorithms
By the end of this lecture you will be able to:
- sort a `DataFrame`
- sort a column with an expression 
- take advantage of fast-track algorithms with `set_sorted`

In this lecture we learn how to sort both on a `DataFrame` and within an expression. We also introduce the fast-track algorithms on sorted data. The fast-track algorithims are optimisations separate from those of the built-in query optimiser. We see how to take advantage of them here.

In [ ]:
import polars as pl

Check out my short youtube video on this topic below

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/oRv1ANrW020?si=LbV4gQpaX--d2106" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

In [ ]:
csv_file = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csv_file)
df.head(3)

We use `pl.Config` to adjust the default so that only 4 rows of a `DataFrame` are printed in this notebook

In [ ]:
pl.Config.set_tbl_rows(4)

## Sorting a `DataFrame`

### Using the `sort` method on `DataFrame`

We can sort a `DataFrame` on a column with the `sort` method

In [ ]:
df.sort("Age")

By default `null` values are at the start of the sort. We can move the `nulls` to the end of the sort by setting the `nulls_last` argument to `True`

In [ ]:
df.sort("Age",nulls_last=True)

We can sort in reverse order with the `descending` argument - note that the `nulls_last` argument is set to the default of `True` so the `null` rows are first

In [ ]:
df.sort("Age",descending=True)

We get the largest values first by setting `nulls_last=True`

In [ ]:
df.sort("Age",descending=True,nulls_last=True)

## Sort on multiple columns
We can sort based on multiple columns with either a list...

In [ ]:
df.sort(["Pclass","Age"])

...or with comma-separated strings

In [ ]:
df.sort("Pclass","Age")

## Sorting a column with an expression

We can transform a column into sorted order within an expression.

In this example we sort the values in every column independent of other columns

In [ ]:
(
    df
    .select(
        pl.all().sort()
    )
)

Within an expression we can also sort all columns with respect to another column using `sort_by`

In [ ]:
(
    df
    .select(
        pl.all().sort_by("Age")
    )
)

It seems like `sort_by` in this case has just replicated the functionality of 
```python
df.sort("Age")
```
However, as we can use `sort_by` in an expression it can be used in other contexts such as in a `groupby` aggregation.  For example, if we wanted to get the name and age of the oldest passenger in each class we can do the following

In [ ]:
pl.Config.set_fmt_str_lengths(100)
(
    df
    .group_by("Pclass")
    .agg(
        pl.col("Name").sort_by("Age").last(),
        pl.col("Age").sort_by("Age").last()
        
    )
)

### Filtering for the largest/smallest values
If we just want to find the largest or smallest values we could do `sort` followed by `head` or `tail`. For example here we find the oldest passengers

In [ ]:
(
    df
    .sort("Age")
    .tail(3)
)

A faster approach is to use `top_k` which does not sort the full `DataFrame` but instead just searches through the rows to filter for the largest/smallest values and then sorts this small subset of rows

In [ ]:
(
    df
    .top_k(
        # Number of records to return
        k=5,
        # Column/expression to sort by
        by="Age",
        # Return the largest records
        descending=False,
        # Ensure the nulls are at the end
        nulls_last=True
    )
)

Some good news: if you do .`sort.head/tail` in lazy mode Polars applies a `top_k` optimization under the hood

In [ ]:
(
    df
    .lazy()
    .sort("Age")
    .tail(3)
    .collect()
)

## Taking advantage of sorted data

For some operations Polars can use a fast track algorithm if it knows the data in a column is sorted.

For example, if we want the `max` value on a sorted column a fast-track algorithm would just take the last (non-`null`) value.

See my blog post for more background on this: https://www.rhosignal.com/posts/polars-loves-sorted-data-1-statistics/

### Checking the sorted status
You can check if Polars **thinks** a column is sorted with the `flags` attribute on a column or a `Series`

In [ ]:
df["PassengerId"].flags

In this case as both the ASC and DESC values are `False` Polars doesn't think the `PassengerID` column is sorted (although we know that is sorted).

You can check the status of all columns at once with the `flags` attribute on a `DataFrame`

In [ ]:
df.flags

We can check if a column is actually sorted with the `is_sorted` method:

In [ ]:
df["PassengerId"].is_sorted()

### Setting the sorted status
If we know that a column is sorted then we can let Polars know using `set_sorted`

In [ ]:
df = (
    pl.read_csv(csv_file)
    .with_columns(
        pl.col("PassengerId").set_sorted()
    )
)
df["PassengerId"].flags

Looking at the output of `flags` we now see `'SORTED_ASC': True`

In the exercises we see the major effect `set_sorted` can have on performance.

If we transform a column with a sorting operation Polars will automatically update the `flags` attribute for that column

In [ ]:
df = (
    pl.read_csv(csv_file)
    .sort("PassengerId")
)
df["PassengerId"].flags

If the data is sorted descending we tell Polars this by passing the `descending` argument:
```python
pl.col("PassengerId").set_sorted(descending=True)
```

### `set_sorted` in an expression
We can use `set_sorted` within an expression. 

For example, if we have a sorted column we can use `set_sorted` to find the `max`

In [ ]:
(
    df
    .select(
        pl.col("PassengerId").set_sorted().max()
    )
)

### Operations with fast-track algorithms
The set of operations that have sorted fast-track algorithms is evolving but includes:
- min
- max
- quantile
- median (a special case of quantile)
- filter
- group_by (see the groupby lectures)
- join (see the join lectures)

## Exercises
In the exercises you will develop your understanding of:
- sorting a `DataFrame`
- sorting in an expression
- using fast-track algorithms with `set_sorted`

## Exercise 1: Sorting a `DataFrame`
Sort the `DataFrame` by whether passengers survived and the alphabetical order of the passenger names.

Configure the output to print 10 lines using `pl.Config`

In [ ]:
pl.Config<blank>
(
    pl.read_csv(csv_file)
)

Add a column for the `familySize` which is the sum of the number of siblings (`SibSp` columns), the number of parents or children (`Parch` columns) plus one for the passenger themself.

Then sort all of the columns by `familySize` inside an expression

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

## Exercise 2: Using `set_sorted`

For this exercise we first create a random array in Numpy and then sort it.

We populate a `DataFrame` with the same array in 2 columns:
- a column that we **will** tell Polars is sorted called `known_sorted`
- a column that we **will not** tell Polars is sorted called `unknown_sorted`

In [ ]:
# Create the sorted array
import numpy as np

N = 10_000_000
sorted_array = np.sort(np.random.standard_normal(N))
# Create the DataFrame
df_sort = pl.DataFrame({"known_sorted": sorted_array, "unknown_sorted": sorted_array})

df_sort.head(3)

Check to see if Polars thinks the `known_sorted` column is sorted yet

Tell polars that the `known_sorted` column is sorted

In [ ]:
df_sort = (
    df_sort
    <blank>
)

Confirm that Polars knows the `known_sorted` column is sorted

Compare how long it takes to find the median of each column. 

> Ignore any you get a message saying that one run took much longer than the others and intermediate values might be cached. This variability in run time is just due to natural variability in runtime. Generally I run it again until i get a stable timing

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    <blank>
)

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    <blank>
)

We want to filter each `DataFrame` to find values less than -2. Compare how long it takes when we apply the filter to the `known_sorted` column compared to the `unknown_sorted` column

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    <blank>
)

The size of the performance difference varies depending on where in the sorted range we are looking for values. To explore this you can comapare performance for e.g. >-2 or <4

You can also vary `N` to see how the difference changes with length.

## Solutions

## Solution to Exercise 1
Sort the `DataFrame` by survival and alphabetical order of the passenger names

In [ ]:
pl.Config.set_tbl_rows(10)
(
    pl.read_csv(csv_file)
    .sort(
        ["Survived","Name"]
    )
)

Add a column for the `familySize` which is the sum of the number of siblings (`SibSp` columns), the number of parents or children (`Parch` columns) plus one for the passenger themself.

Then sort all of the columns by `familySize` inside an expression

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns( 
        (
            pl.col('SibSp') + pl.col('Parch') + 1
        ).alias('familySize')
    )
    .select(
        pl.all().sort_by("familySize")
    )
)

## Solution to exercise 2


Create the `DataFrame`

In [ ]:
import numpy as np

N = 10_000_000
sorted_array = np.sort(np.random.standard_normal(N))
df_sort = pl.DataFrame({"known_sorted": sorted_array, "unknown_sorted": sorted_array})

df_sort.head(3)

Check to see if Polars thinks the `known_sorted` column is sorted yet

In [ ]:
df_sort["known_sorted"].flags

Tell polars that the `known_sorted` column is sorted

In [ ]:
df_sort = (
    df_sort
    .with_columns(
        pl.col('known_sorted').set_sorted()
    )
)

Confirm that Polars knows the `known_sorted` column is sorted

In [ ]:
df_sort["known_sorted"].flags

Compare how long it takes to find the median of each column.  Ignore any you get a message saying that one run took much longer than the others and intermediate values might be cached. This variability in run time is just something that can happen

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    .select(
        pl.col("known_sorted").median()
    )
)

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    .select(
        pl.col("unknown_sorted").median()
    )
)

We want to filter the `DataFrame` by the `known_sorted` and `unknown_sorted` columns to find values less than -2. Compare how long it takes when we apply the filter to the `known_sorted` column compared to the `unknown_sorted` column

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    .filter(
        pl.col("known_sorted") < -2
    )
)

In [ ]:
%%timeit -n1 -r5
(
    df_sort
    .filter(
        pl.col("unknown_sorted") < -2
    )
)